##### Copyright 2021 The Cirq Developers

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Calibrations
The [tutorial on calibration](https://quantumai.google/cirq/google/calibration) gives an introduction to calibration metrics and how to retrieve them from the Google Quantum Computing Service (QCS). In this tutorial, we show how to visualize these calibration metrics via single qubit heatmaps, two qubit interaction heatmaps and integrated histograms.

**Disclaimer:** The calibration metrics of the quantum processor shown in the tutorial do not correspond to actual production metrics.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/tutorials/google/visualizing_calibration_metrics>"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/tutorials/google/visualizing_calibration_metrics.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/visualizing_calibration_metrics.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/tutorials/google/visualizing_calibration_metrics.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
#@title Necessary Imports
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet --pre cirq
    print("installed cirq.")
    import cirq
import os
from random import random
from itertools import combinations
import matplotlib.pyplot as plt

## Authenticate and install Cirq

For details of authentication and installation, please see [go/quantum-engine-quickstart](http://go/quantum-engine-quickstart).

Note:  The below code will install the latest stable release of cirq.  If you need the latest and greatest features and don't mind if a few things aren't quite working correctly, you can install the pre-release version of `cirq` using `pip install --pre cirq` instead of `pip install cirq` to get the most up-to-date features of cirq.

1. Enter the Cloud project ID you'd like to use in the `YOUR_PROJECT_ID` field.
2. Then run the cell below (and go through the auth flow for access to the project id you entered).

<img src="https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/images/run-code-block.png" alt="Quantum Engine console">

In [ ]:
#@title GCloud Authentication 
# The Google Cloud Project id to use.
YOUR_PROJECT_ID = '' #@param {type:"string"}
if YOUR_PROJECT_ID == '':
    if 'GOOGLE_CLOUD_PROJECT' not in os.environ:
        raise Exception("Please setup project_id in this cell or set the `GOOGLE_CLOUD_PROJECT` env var to your project id.")
    YOUR_PROJECT_ID = os.environ['GOOGLE_CLOUD_PROJECT']
else:
    os.environ['GOOGLE_CLOUD_PROJECT'] = YOUR_PROJECT_ID

def authenticate_user():
  """Runs the user through the Colab OAuth process.

  Checks for Google Application Default Credentials and runs interactive login
  if the notebook is executed in Colab. In case the notebook is executed in Jupyter notebook
  or other IPython runtimes, no interactive login is provided, it is assumed that the
  `GOOGLE_APPLICATION_CREDENTIALS` env var is set or `gcloud auth application-default login`
  was executed already.

  For more information on using Application Default Credentials see
  https://cloud.google.com/docs/authentication/production
  """
  in_colab = False
  try:
     from IPython import get_ipython
     in_colab = 'google.colab' in str(get_ipython())
  except:
     # Notebook is not executed within IPython. Assuming external authentication.
     return

  if in_colab:
      from google.colab import auth
      print("Getting OAuth2 credentials.")
      print("Press enter after entering the verification code.")
      auth.authenticate_user(clear_output=False)
      print("Authentication complete.")
  else:
      print("Notebook is not executed with Colab, assuming Application Default Credentials are setup.")

authenticate_user()

print("Successful authentication to Google Cloud.")

In [ ]:
# @title Quantum Engine Configuration
# @markdown Replace the `PROCESSOR_ID` and `YOUR_PROJECT_ID` with your details.
PROCESSOR_ID = 'weber' #@param {type: 'string'}
cals = cirq.google.get_engine_calibration(PROCESSOR_ID, YOUR_PROJECT_ID)

In [ ]:
#@title View available calibration metrics
#@markdown See https://quantumai.google/cirq/google/calibration for a description of the available calibration metrics
for k, v in cals.items():
  print(k)

# Metrics

## Single Qubit Isolated Benchmarking

### Average Error


In [ ]:
fig, _ = cals.plot('single_qubit_rb_average_error_per_gate', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Incoherent Error 


In [ ]:
fig, _ = cals.plot('single_qubit_rb_incoherent_error_per_gate', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Pauli Error

In [ ]:
fig, _ = cals.plot('single_qubit_rb_pauli_error_per_gate', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Comparison Histograms

In [ ]:
keys = ['single_qubit_rb_pauli_error_per_gate', 'single_qubit_rb_average_error_per_gate', 'single_qubit_rb_incoherent_error_per_gate']
fig, ax = plt.subplots(figsize=(16, 8))
cals.plot_histograms(keys, ax)
ax.set_title("Single Qubit Error Comparison Histograms")
fig.show()

## Single Qubit Isolated Readout


### Zero State Errors

In [ ]:
fig, _ = cals.plot('single_qubit_p00_error', fig=plt.figure(figsize=(20, 10)))
fig.show()

### One State Errors

In [ ]:
fig, _ = cals.plot('single_qubit_p11_error', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Readout Separation Error

In [ ]:
fig, _ = cals.plot('single_qubit_readout_separation_error', fig=plt.figure(figsize=(20, 10)))
fig.show()

## Parallel Readout Error

### Parallel P00 Error

In [ ]:
fig, _ = cals.plot('parallel_p00_error', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Parallel P11 Error

In [ ]:
fig, _ = cals.plot('parallel_p11_error', fig=plt.figure(figsize=(20, 10)))
fig.show()

## Isolated Pair XEB, sqrt(iswap)

### Average Error

In [ ]:
fig, _ = cals.plot('two_qubit_sqrt_iswap_gate_xeb_average_error_per_cycle', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Incoherent Error

In [ ]:
fig, _ = cals.plot('two_qubit_sqrt_iswap_gate_xeb_incoherent_error_per_cycle', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Pauli Error

In [ ]:
fig, _ = cals.plot('two_qubit_sqrt_iswap_gate_xeb_pauli_error_per_cycle', fig=plt.figure(figsize=(20, 10)))
fig.show()

## Parallel Pair XEB, sqrt(iswap)

### Average Error

In [ ]:
fig, _ = cals.plot('two_qubit_parallel_sqrt_iswap_gate_xeb_average_error_per_cycle', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Incoherent Error

In [ ]:
fig, _ = cals.plot('two_qubit_parallel_sqrt_iswap_gate_xeb_incoherent_error_per_cycle', fig=plt.figure(figsize=(20, 10)))
fig.show()

### Pauli Error

In [ ]:
fig, _ = cals.plot('two_qubit_parallel_sqrt_iswap_gate_xeb_pauli_error_per_cycle', fig=plt.figure(figsize=(20, 10)))
fig.show()